<a href="https://colab.research.google.com/github/VSubhash1/Intent-Classification-/blob/main/Intent_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch sentencepiece speechrecognition pyaudio gTTS gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [ ]:
!apt-get update
!apt-get install -y portaudio19-dev
!pip install pyaudio

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,274 kB]
Hit:13 https://ppa.launchpadcontent.net/

In [ ]:
!pip install transformers datasets torch sentencepiece gradio

In [ ]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import gradio as gr
import os


In [ ]:
intents = {
    "intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
         "responses": ["Hello!", "Hi there!", "Hey!"]},

        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "See ya"],
         "responses": ["Goodbye!", "Take care!", "See you!"]},

        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "Much appreciated"],
         "responses": ["You're welcome!", "No problem!", "Glad to help!"]},

        {"tag": "order_pizza",
         "patterns": ["I want a pizza", "Can I order pizza?", "Pizza delivery"],
         "responses": ["Sure! What toppings?", "Ordering pizza now!"]}
    ]
}

sentences, labels, label2id, id2label = [], [], {}, {}
for i, intent in enumerate(intents["intents"]):
    tag = intent["tag"]
    label2id[tag] = i
    id2label[i] = tag
    for p in intent["patterns"]:
        sentences.append(p)
        labels.append(i)

print("Classes:", label2id)

Classes: {'greeting': 0, 'goodbye': 1, 'thanks': 2, 'order_pizza': 3}


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=32):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding="max_length",
                                  max_length=self.max_len, return_tensors="pt")
        return { "input_ids": encoding["input_ids"].squeeze(),
                 "attention_mask": encoding["attention_mask"].squeeze(),
                 "labels": torch.tensor(label) }

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42)
train_dataset = IntentDataset(X_train, y_train, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id))
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

for epoch in range(2):  # demo training
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 1.4296
Epoch 2, Loss: 1.3555


In [ ]:
def predict_intent(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs, dim=1).item()
    return id2label[pred_label], float(probs[0][pred_label])

def classify_intent(user_input):
    intent, conf = predict_intent(user_input)
    return f"Predicted intent: {intent}\nConfidence: {conf:.2f}"

In [ ]:
def voice_to_text(file_path):
    import speech_recognition as sr
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio)
    intent, conf = predict_intent(text)
    return f"You said: {text}\nPredicted intent: {intent}\nConfidence: {conf:.2f}"

In [ ]:
def tts_response(user_input):
    from gtts import gTTS
    intent, conf = predict_intent(user_input)
    response_text = f"Predicted intent is {intent} with confidence {conf:.2f}"
    tts = gTTS(response_text)
    filename = "response.mp3"
    tts.save(filename)
    return filename

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🎤 Intent Classifier with BERT (Text + Voice)")

    with gr.Row():
        text_in = gr.Textbox(label="Type your message")
        text_out = gr.Textbox(label="Predicted Intent")
    btn1 = gr.Button("Classify Text")
    btn1.click(classify_intent, inputs=text_in, outputs=text_out)

    gr.Markdown("Or speak your message 👇")
    mic_in = gr.Audio(type="filepath", label="🎤 Speak or Upload Audio")
    voice_out = gr.Textbox(label="Predicted Intent (from Voice)")
    btn2 = gr.Button("Classify Voice")
    btn2.click(voice_to_text, inputs=mic_in, outputs=voice_out)

    gr.Markdown("🔊 Get spoken response")
    tts_in = gr.Textbox(label="Type message for spoken prediction")
    tts_out = gr.Audio(label="Voice Output")
    btn3 = gr.Button("Speak Prediction")
    btn3.click(tts_response, inputs=tts_in, outputs=tts_out)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d837b66e6bf7101539.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
git remote add origin git@github.com:VSubhash1/Intent-Classification-.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (ipython-input-3390904513.py, line 1)